In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import dask
import os, sys
import glob
import zarr
from joblib import Parallel, delayed
import os
import dask.array as da
import matplotlib.pyplot as plt
import metpy

root_dir = '/home/harish/Desktop/Modeling-Frontal-Low-Level-Jets-and-Associated-Extreme-Wind-Ramps/' if os.path.exists('/home/harish/Desktop/Modeling-Frontal-Low-Level-Jets-and-Associated-Extreme-Wind-Ramps/') else '/media/ssd_4tb_qvo/Modeling-Frontal-Low-Level-Jets-and-Associated-Extreme-Wind-Ramps/'

from data_processing.libraries import *

/home/harish/Desktop/Modeling-Frontal-Low-Level-Jets-and-Associated-Extreme-Wind-Ramps/data_processing/libraries.py:151: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(f"{root_dir}/windturbines.txt", delim_whitespace=True,header=None)
/home/harish/Desktop/Modeling-Frontal-Low-Level-Jets-and-Associated-Extreme-Wind-Ramps/data_processing/libraries.py:156: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  power_curve_type_1 = pd.read_csv(f"{root_dir}/wind-turbine-1.tbl",delim_whitespace=True,skiprows=2,header=None,usecols=[0,2])
/home/harish/Desktop/Modeling-Frontal-Low-Level-Jets-and-Associated-Extreme-Wind-Ramps/data_processing/libraries.py:157: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` ins

In [2]:
# we would like to make sure that the reference height levels cover the profiler levels and NOW23 levels, including the 0 m level.
profilers_levels = np.array([10] + list(range(100, 501, 25)))
NOW23_levels = np.array([10] + list(range(20, 301, 20)) + [400, 500])
CERRA_levels = np.array([10,15, 30, 50., 75., 100., 150., 200., 250., 300., 400., 500])
# combine the two arrays and remove duplicates
ref_H = np.unique(np.concatenate((np.array([0]),profilers_levels, NOW23_levels, CERRA_levels)))

# These variables are used while creating the Chebyshev coefficients
poly_order = 4
CPtype = 1

def normalize(H,ref_H=ref_H):
    '''
    Normalizes the height levels between -1 and 1
    ref_H: A vector of reference height levels, in our case CERRA levels
    H: A vector of height levels
    '''
    a = 2 / (np.max(ref_H) - np.min(ref_H))
    b = - (np.max(ref_H) + np.min(ref_H)) / (np.max(ref_H) - np.min(ref_H))
    Hn = a * ref_H + b

    Hn = np.interp(H, ref_H, Hn)
    return Hn

def Chebyshev_Basu(x, poly_order=poly_order, CPtype=CPtype):
    '''
    This function computes the Chebyshev polynomials, according to the equations in Mason, J. C., & Handscomb, D. C. (2002). Chebyshev polynomials. Chapman and Hall/CRC.
    x: input variable, between -1 and 1
    poly_order: order of polynomials
    CPtype: 1 or 2, according to the publication
    '''
    if x.ndim == 1:
        x = x[:, np.newaxis]

    CP = np.zeros((len(x), poly_order + 1))

    CP[:, 0] = 1  # T0(x) = 1
    if poly_order >= 1:
        if CPtype == 1:  # Chebyshev polynomial of first kind
            CP[:, 1] = x.flatten()  # T1(x) = x
        else:  # Chebyshev polynomial of second kind
            CP[:, 1] = 2 * x.flatten()  # T1(x) = 2x
        if poly_order >= 2:
            for n in range(2, poly_order + 1):
                CP[:, n] = 2 * x.flatten() * CP[:, n - 1] - CP[:, n - 2]
    return CP

def Chebyshev_Coeff(H, U,poly_order=poly_order,CPtype=CPtype,ref_H=ref_H):
    '''
    This function computes the Chebyshev coefficients through inverse transform of system of linear equations
    H: height levels, in their useual units
    U: wind speed at the height levels
    p: polynomial order
    CPtype: 1 or 2, according to the publication
    '''
    H = H.flatten()
    U = U.flatten()

    # Normalize H
    Hn = normalize(H, ref_H=ref_H)
    
    # Remove NaN values
    Indx = np.where(~np.isnan(U))[0]
    Ha = Hn[Indx]
    Ua = U[Indx]
    N = len(Ua)

    # Linearly extrapolate wind values at the boundaries
    #spline_left = interp1d(Ha, Ua, kind='linear', fill_value='extrapolate')
    #Uax = spline_left([-1])

    #spline_right = interp1d(Ha, Ua, kind='linear', fill_value='extrapolate')
    #Uay = spline_right([1])
    #Ua = np.concatenate([Uax, Ua, Uay])
    #Ha = np.concatenate([-1 + np.zeros(1), Ha, 1 + np.zeros(1)])       # these two seems are unnecessary, which bring adidtional offset due to extrapolation.
    
    # Predict the gap-filled and denoised profile
    PL = Chebyshev_Basu(Ha, poly_order=poly_order, CPtype=CPtype)
    # Compute the coefficients C
    Coeff = np.linalg.pinv(PL) @ Ua
    return Coeff

def WindProfile(Z,Coeff, poly_order=poly_order, CPtype=CPtype,ref_H=ref_H):
    '''
    This function computes the full level wind profile provided vertical levels and the Chebyshev coefficients
    Z: height levels, in their useual units
    Coeff: Chebysev coefficients
    '''
    # Normalize H
    Hn = normalize(Z, ref_H=ref_H)
    PL_full = Chebyshev_Basu(Hn, poly_order=poly_order, CPtype=CPtype)
    Mp = PL_full @ Coeff
    return Mp

def chebyshev(z,y: np.ndarray) -> np.ndarray:
    """Calculate Chebyshev coefficients for input y"""
    return Chebyshev_Coeff(z, y)


def chebyshev_vec(y: xr.DataArray, dim: str) -> xr.DataArray:
    """
    Calculate Chebyshev coefficients for input y in `dim` direction.
    Sample usage cheb = chebyshev_vec(ds, dim="heightAboveGround")
    """
    cheb = xr.apply_ufunc(
        lambda y_i: xr.DataArray(chebyshev(y_i), dims=["coeff"]),  # we need to make the output a DataArray with new dim
        y,  # entire dataset as input
        input_core_dims=[[dim]],  # dimension along which to apply chebyshev
        output_core_dims=[["coeff"]],  # dimension of the output, which is the coefficients
        vectorize=True,  # vectorize the operation
        dask="parallelized",
        dask_gufunc_kwargs={"output_sizes": {"coeff": poly_order+1}},
    )
    cheb.name = "Chebyshev_coefficients"
    return cheb

# CERRA wind power

In [3]:
CERRA_Cheybshev_zarr_store = '/data/harish/CERRA_wind_profiles_and_Chebyshev_coefficients/CERRA_Chebyshev_coefficients.zarr'
CERRA_coordinates = xr.open_dataset('/home/harish/Ongoing_Research/CERRA_wind_profiles_and_Chebyshev_coefficients/CERRA_coordinates.nc')
def find_nearest_indice(ds_lat,ds_lon,target_lat=None, target_lon=None):
    '''
    ds_lat: xarray DataArray of latitude
    ds_lon: xarray DataArray of longitude
    target_lat: float, target latitude
    target_lon: float, target longitude
    returns indices: tuple of indices of the nearest grid point
    '''
    distance_squared = (ds_lat - target_lat)**2 + (ds_lon - target_lon)**2
    indices = np.unravel_index(np.nanargmin(distance_squared), distance_squared.shape)
    #print(f'Closest indices in the order of latitude (y) and longitude (x) are : {indices}')
    return indices

In [ ]:
indices = [
    find_nearest_indice(CERRA_coordinates.latitude, CERRA_coordinates.longitude, lat.item(), lon.item())
    for lat, lon in zip(turbine_lats, turbine_lons)
]
# Split the list of tuples into separate lists for y and x indices
y_indices, x_indices = zip(*indices)
sample_ys = xr.DataArray(np.array(y_indices), dims="points")
sample_xs = xr.DataArray(np.array(x_indices), dims="points")

In [188]:
for case in [1,2,3,4,5]:
    time_range = ramp_periods[case-1]
    ds = xr.open_zarr(CERRA_Cheybshev_zarr_store).Chebyshev_coefficients.isel(y=sample_ys,x=sample_xs).sel(time=slice(*time_range)).assign_coords(turbine_type=("points", turbine_types), 
                      turbine_lat=("points", turbine_lats), 
                      turbine_lon=("points", turbine_lons),
                      hub_heights=("points", [hub_heights[i-1] for i in turbine_types]))
    
    # Apply the function along the 'time' dimension
    ws = xr.apply_ufunc(
        lambda z_i,y_i: xr.DataArray(WindProfile([z_i],y_i)),# dims=["heightAboveGround"]),  
        ds.hub_heights,  # Pass the height levels as an argument
        ds.load(),  # Input dataset
        input_core_dims=[[],["coeff"]],  # Expect features to be the core dimension
        #output_core_dims=[["heightAboveGround"]],  # Output will have a new height dimension
        vectorize=True,  # Ensures element-wise operation
    )
    tp = []
    for i in range(ws.points.size):
        turbine_type = ws.turbine_type[i].item()
        ws_i = ws.sel(points=i)
        tp.append(turbine_power(ws_i, power_curves[turbine_type - 1].values))
    tp = xr.concat(tp, dim="points")
    tp = tp.sum(dim="points")
    
    target_dir = os.path.join(root_dir, f'WES_dataset/CERRA/FLLJ_{case}')
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    tp.to_netcdf(os.path.join(target_dir, 'turbine_power.nc'))

# ERA5 wind power

In [21]:
turbine_sites = xr.Dataset(
{"latitude": ("points", turbine_lats), "longitude": ("points", turbine_lons)}
)

for case in [2,3,4,5]:
    time_range = ramp_periods[case-1]
    ds = xr.open_zarr(
        'gs://gcp-public-data-arco-era5/ar/model-level-1h-0p25deg.zarr-v1',
        chunks=None,
        storage_options=dict(token='anon'),
    )

    lu = find_nearest_indice(ds.latitude, ds.longitude, turbine_lats.max(),turbine_lons.min())
    rd = find_nearest_indice(ds.latitude, ds.longitude, turbine_lats.min(),turbine_lons.max())

    ds = ds[['u_component_of_wind', 'v_component_of_wind','geopotential']].sel(
        hybrid=slice(120,None),
        time=slice(*time_range),
        latitude=slice(ds.latitude[lu[0]],ds.latitude[rd[0]]),
        longitude=slice(ds.longitude[lu[1]],ds.longitude[rd[1]])).load()
    ds = ds.isel(hybrid=slice(None,None,-1))

    ws = np.sqrt(ds.u_component_of_wind**2 + ds.v_component_of_wind**2)
    ds_ = xr.Dataset(
        {
            "ws": ws,
            "geopotential": ds.geopotential / 9.81,
        }
    )
    ds_ = ds_.where(ds_.geopotential <= 500,drop=True)

    chebyshev = xr.apply_ufunc(
        lambda z_i,y_i: xr.DataArray(Chebyshev_Coeff(z_i,y_i)),# dims=["heightAboveGround"]),  
        ds_.geopotential,  # Pass the height levels as an argument
        ds_.ws,  # Input dataset
        input_core_dims=[['hybrid'],["hybrid"]],  # Expect features to be the core dimension
        output_core_dims=[["coeff"]],  # Output will have a new height dimension
        vectorize=True,  # Ensures element-wise operation
        dask="parallelized",
        dask_gufunc_kwargs={"output_sizes": {"coeff": poly_order+1}},
    )

    chebyshev = chebyshev.sel(latitude=turbine_sites.latitude, longitude=turbine_sites.longitude, method="nearest").assign_coords(turbine_type=("points", turbine_types), 
                        turbine_lat=("points", turbine_lats), 
                        turbine_lon=("points", turbine_lons),
                        hub_heights=("points", [hub_heights[i-1] for i in turbine_types]))
    
    # Apply the function along the 'time' dimension
    ws = xr.apply_ufunc(
        lambda z_i,y_i: xr.DataArray(WindProfile([z_i],y_i)),# dims=["heightAboveGround"]),  
        chebyshev.hub_heights,  # Pass the height levels as an argument
        chebyshev.load(),  # Input dataset
        input_core_dims=[[],["coeff"]],  # Expect features to be the core dimension
        #output_core_dims=[["heightAboveGround"]],  # Output will have a new height dimension
        vectorize=True,  # Ensures element-wise operation
    )

    tp = []
    for i in range(ws.points.size):
        turbine_type = ws.turbine_type[i].item()
        ws_i = ws.sel(points=i)
        tp.append(turbine_power(ws_i, power_curves[turbine_type - 1].values))
    tp = xr.concat(tp, dim="points")
    tp = tp.sum(dim="points")

    target_dir = os.path.join(root_dir, f'WES_dataset/ERA5/FLLJ_{case}')
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    tp.to_netcdf(os.path.join(target_dir, 'turbine_power.nc'))